In [1]:
import numpy as np
import pandas as pd

In [2]:
import glob

# Importing drive method from colab for accessing google drive
from google.colab import drive

In [4]:
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
from numpy import load


In [6]:
testimages = load(r'/content/drive/My Drive/DATASET/numpyarr/testimg.npy')



In [7]:
trainimages=load(r'/content/drive/My Drive/DATASET/numpyarr/trainimg.npy')

In [8]:
validationimages=load(r'/content/drive/My Drive/DATASET/numpyarr/valimg.npy')
testlabels=load(r'/content/drive/My Drive/DATASET/numpyarr/testlab.npy')
trainlabels=load(r'/content/drive/My Drive/DATASET/numpyarr/trainlab.npy')
validationlabels=load(r'/content/drive/My Drive/DATASET/numpyarr/vallab.npy')

In [9]:
np.shape(trainimages)

(8029, 112, 150, 3)

In [10]:
from tensorflow import keras

In [11]:
model1 = keras.models.load_model(r'/content/drive/My Drive/DATASET/archs/model-final-inception.h5')

In [12]:
model2= keras.models.load_model(r'/content/drive/My Drive/DATASET/archs/model-final-VGG16.h5')

In [26]:
model3=keras.models.load_model(r'/content/drive/My Drive/DATASET/archs/model-final-resnet.h5')

In [27]:
model4=keras.models.load_model(r'/content/drive/My Drive/DATASET/archs/model-final.h5')

In [15]:
from deepstack.base import KerasMember

In [16]:
member1 = KerasMember(name="model2", keras_model=model2,   train_batches=(trainimages,trainlabels), val_batches=(validationimages, validationlabels))

In [17]:
member2 = KerasMember(name="model1", keras_model=model1, train_batches=(trainimages,trainlabels), val_batches=(validationimages, validationlabels))

In [29]:
member3 = KerasMember(name="model3", keras_model=model3, train_batches=(trainimages,trainlabels), val_batches=(validationimages, validationlabels))

In [36]:
member1.name="InceptionV3"
member2.name="VGG16"
member3.name="ResNet 50"

In [37]:
from deepstack.ensemble import DirichletEnsemble

wAvgEnsemble = DirichletEnsemble()
wAvgEnsemble.add_members([member1, member2, member3])
wAvgEnsemble.fit()
wAvgEnsemble.describe()

InceptionV3 - Weight: 0.3560 - roc_auc_score: 0.9039
VGG16 - Weight: 0.1029 - roc_auc_score: 0.8779
ResNet 50 - Weight: 0.5412 - roc_auc_score: 0.9295
DirichletEnsemble roc_auc_score: 0.9510


In [38]:
from deepstack.ensemble import StackEnsemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier

stack = StackEnsemble()
stack.model = ExtraTreesClassifier(verbose=0, n_estimators=200, 
                                  max_depth=15, n_jobs=20, min_samples_split=20)
stack.add_members([member1, member2,member3])
stack.fit()
stack.describe(metric= accuracy_score)



Calling predict
InceptionV3 - accuracy_score: 0.9039
VGG16 - accuracy_score: 0.8779
ResNet 50 - accuracy_score: 0.9295
StackEnsemble accuracy_score: 0.8607


0.8607021517553793

In [39]:
from deepstack.ensemble import StackEnsemble
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

#Ensure you have the scikit-learn version >= 0.22 installed
print("sklearn version must be >= 0.22. You have:", sklearn.__version__)

stack = StackEnsemble()

# 2nd Level Meta-Learner
estimators = [
    ('rf', RandomForestClassifier(verbose=0, n_estimators=200, max_depth=15, n_jobs=20, min_samples_split=30)),
    ('etr', ExtraTreesClassifier(verbose=0, n_estimators=200, max_depth=10, n_jobs=20, min_samples_split=20))
]
# 3rd Level Meta-Learner
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

stack.model = clf
stack.add_members([member1, member2,member3])
stack.fit()
stack.describe(metric=sklearn.metrics.accuracy_score)




sklearn version must be >= 0.22. You have: 0.22.2.post1
Calling predict
InceptionV3 - accuracy_score: 0.9039
VGG16 - accuracy_score: 0.8779
ResNet 50 - accuracy_score: 0.9295
StackEnsemble accuracy_score: 0.8607


0.8607021517553793

In [33]:
pip install deepstack==0.0.9